# Trabajo Práctico N°3


    img_align_celeba.zip: All the face images, cropped and aligned
    list_eval_partition.csv: Recommended partitioning of images into training, validation, testing sets. Images 1-162770 are training, 162771-182637 are validation, 182638-202599 are testing
    list_bbox_celeba.csv: Bounding box information for each image. "x_1" and "y_1" represent the upper left point coordinate of bounding box. "width" and "height" represent the width and height of bounding box
    list_landmarks_align_celeba.csv: Image landmarks and their respective coordinates. There are 5 landmarks: left eye, right eye, nose, left mouth, right mouth
    list_attr_celeba.csv: Attribute labels for each image. There are 40 attributes. "1" represents positive while "-1" represents negative


In [ ]:
# un poco menos de warnings de tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# de python, para especificar rutas de archivos y directorios
from pathlib import Path

# lib para trabajar con arrays
import numpy as np
import pandas as pd

# lib que usamos para mostrar las imágenes
import matplotlib.pyplot as plt

# libs que usamos para construir y entrenar redes neuronales, y que además tiene utilidades para leer sets de 
# imágenes
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications import VGG16
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

# libs que usamos para tareas generales de machine learning. En este caso, métricas
from sklearn.metrics import accuracy_score, confusion_matrix

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [ ]:
#Imagenes_DIR = Path('./dataset_TP3/img_align_celeba')
IMG_PATH = Path('/home/matias/Documentos/archive/img_align_celeba/img_align_celeba')
atributos_DIR = Path('/home/matias/Documentos/archive/list_attr_celeba.csv')
set_DIR = Path('/home/matias/Documentos/archive/list_eval_partition.csv')

da = pd.read_csv(atributos_DIR, usecols=['image_id','No_Beard'])
dd = pd.read_csv(set_DIR)
df=da.merge(dd, on="image_id", how="left")
df.head(20)

In [ ]:
# Reset the columns values to categorical./
df.loc[df['No_Beard'] == -1,'No_Beard'] = "Barba"
df.loc[df['No_Beard'] == 1,'No_Beard'] = "No_Barba"

train = df.loc[df['partition'] == 0]
validation = df.loc[df['partition'] == 1]
test= df.loc[df['partition'] == 2]

train=train.drop(['partition'],axis=1)
validation=validation.drop(['partition'],axis=1)
test=test.drop(['partition'],axis=1)

train.head(20)

# Análisis exploratorio del conjunto de datos

Aclarar que archivos hay en el dataset, cuales usamos y como juntamos los datos

### Distribución de la variable

### General

In [ ]:
category_count = df["No_Beard"].value_counts()
print(category_count)

higher_category = list(category_count.index)[0]

In [ ]:
df["No_Beard"].value_counts().plot.bar()

In [ ]:
atributos=np.array(df)
atributos[148010][1] # uno menos 

#hacer labels con lo de los atributos.no_beard
#hacer x_train(imagenes, tamaño, tamaño)
#hacer y_train(atributos.no_beard)

### Por partición

### Estructura y tipo de imagen

In [ ]:
from PIL import Image

In [ ]:
img_path = '/home/matias/Documentos/archive/img_align_celeba/img_align_celeba/000002.jpg'  # Reemplaza con la ruta de tu imagen
img = Image.open(img_path)

In [ ]:
width, height = img.size

In [ ]:
print(f'Ancho: {width} píxeles')
print(f'Alto: {height} píxeles')

In [ ]:
img_array = np.array(img)
depth = img_array.shape[2]
print(f'Profundidad de la imagen: {depth}')

Según lo visto anteriormente concluimos que poe cuestiones de tiempo de entrenamiento vamos a aplicar los siguientes cambios a las imagenes al armar los conjuntos de entrenaminto

In [ ]:
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 100

train_datagen = ImageDataGenerator(
    rescale=1./255,
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_dataframe(
    train, 
    '/home/matias/Documentos/archive/img_align_celeba/img_align_celeba'+'/', 
    x_col='image_id',
    y_col='No_Beard',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    #validate_filenames=False
)

validation_generator = validation_datagen.flow_from_dataframe(
    validation, 
    '/home/matias/Documentos/archive/img_align_celeba/img_align_celeba'+'/', 
    x_col='image_id',
    y_col='No_Beard',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    #validate_filenames=False
)

test_generator = validation_datagen.flow_from_dataframe(
    test, 
    '/home/matias/Documentos/archive/img_align_celeba/img_align_celeba'+'/', 
    x_col='image_id',
    y_col='No_Beard',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    #validate_filenames=False
)

In [ ]:
def sample_images(dataset):
    plt.figure(figsize=(10, 10))
    images, labels = dataset.next()
    for i in range(10):
        ax = plt.subplot(6,6,i+1)
        plt.imshow(images[i])
        plt.title(labels[i])
        #print(images[17][26])
        
        #print(labels[i])
        #print(i)
        #clase=atributos[i][26]
        #plt.title(class_name[clase])
        plt.axis("off")


In [ ]:
sample_images(train_generator)

### Entrenamiento de modelos

In [ ]:
base_vgg_model = tf.keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, input_shape= IMAGE_SIZE + (3,))

In [ ]:
base_vgg_model.trainable = False
base_vgg_model.summary()


In [ ]:
vgg_model = Sequential(
    [
    base_vgg_model,
    Flatten(),
    Dense(256,activation='relu'),
    Dense(256,activation='relu'),
    Dense(64,activation='relu'),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='sigmoid')
]
)

In [ ]:
vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=4, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.001)
callbacks = [earlystop, learning_rate_reduction]

history = vgg_model.fit(train_generator, validation_data = validation_generator
                        , validation_steps=len(validation_df)//BATCH_SIZE
                        ,steps_per_epoch=len(train_df)//BATCH_SIZE,
                        epochs=10, verbose = 1, callbacks=callbacks)



In [ ]:
""""""
pretrained_model = VGG16(include_top=False, input_shape= IMAGE_SIZE + (3,))
pretrained_model.trainable = False

vgg16_model = Sequential(
    [
    pretrained_model,
    #Convolution2D(filters=8, kernel_size=(2, 2), strides=1, activation='tanh'),
    #MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(50,activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')
]
)

vgg16_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy',],
)

In [ ]:
model_weights_at_epochs = {}

class OurCustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        model_weights_at_epochs[epoch] = self.model.get_weights()
        
history = vgg16_model.fit(
    train_generator, validation_data = validation_generator
                        , validation_steps=len(validation)//BATCH_SIZE
                        ,steps_per_epoch=len(train)//BATCH_SIZE,
                        epochs=10, verbose = 1, callbacks=[OurCustomCallback()]
)

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
train, test = fashion_mnist.load_data()
(x_train, y_train) = train 
(x_test, y_test) = test

In [ ]:
train, test = fashion_mnist.load_data()
(x_train, y_train) = train 
(x_test, y_test) = test

In [ ]:
y_train[1]

In [ ]:
x_train[1]

In [ ]:
train_generator.head()

In [ ]:
train_generator = validation_datagen.flow_from_dataframe(
    train, 
    '/home/matias/Documentos/archive/img_align_celeba/img_align_celeba'+'/', 
    x_col='image_id',
    y_col='No_Beard',
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=BATCH_SIZE,
    #validate_filenames=False
)

data_array, labels_array = train_generator.next()
    
data_array = []
labels_array = []

for _ in range(BATCH_SIZE):
    batch_data, batch_labels = train_generator.next()
    data_array.append(batch_data)
    labels_array.append(batch_labels)

data_array = np.concatenate(data_array)
labels_array = np.concatenate(labels_array)


In [ ]:
labels_array[0]

In [ ]:
# función para mostrar imágenes
def mostrar_imagenes(entradas, salidas):
    plt.figure(figsize=(10,10))
    for i in range(10):
        plt.subplot(6,6,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(entradas[i])
        plt.xlabel(salidas[i])
    plt.show()

mostrar_imagenes(data_array, labels_array)


In [ ]:
data_array[0]

In [ ]:
train_generator[0][0]